# "kaggle - Geospatial Analysis04"

# 4. Exercise: Manipulating Geospatial Data

소개
당신은 Starbucks Reserve Roastery의 다음 매장을 찾고 있는 Starbucks 빅 데이터 분석가입니다. 이 로스터리는 일반적인 스타벅스 매장보다 훨씬 크며 고급 라운지 공간과 함께 다양한 음식과 와인 옵션을 비롯한 몇 가지 추가 기능을 갖추고 있습니다. 캘리포니아 주에 있는 여러 카운티의 인구 통계를 조사하여 잠재적으로 적합한 위치를 결정합니다.

In [50]:
import math
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim            # What you'd normally run
#from learntools.geospatial.tools import Nominatim # Just for this exercise

import folium 
from folium import Marker
from folium.plugins import MarkerCluster

이전 연습의 embed_map() 함수를 사용하여 지도를 시각화합니다.

In [46]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

#### 4.1 누락된 위치를 지오코딩합니다.
다음 코드 셀을 실행하여 캘리포니아 주에 있는 Starbucks 위치를 포함하는 DataFrame 스타벅스를 만듭니다.

In [47]:
# Load and preview Starbucks locations in California
starbucks = pd.read_csv("C:/Users/Kangdaeyong/Desktop/datamining/kaggle_geospatial_analysis/archive/starbucks_locations.csv")
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


대부분의 상점은 (위도, 경도) 위치를 알고 있습니다. 그러나 버클리시의 모든 위치가 누락되었습니다.

In [48]:
# How many rows in each column have missing values?
print(starbucks.isnull().sum())

# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


아래 코드 셀을 사용하여 Nominatim 지오코더로 이 값을 채우십시오.

튜토리얼에서 우리는 값을 지오코딩하기 위해 Nominatim()(geopy.geocoders에서)을 사용했으며 이것은 이 과정 이외의 자체 프로젝트에서 사용할 수 있는 것입니다.

이 연습에서는 약간 다른 함수 Nominatim()을 사용합니다(learntools.geospatial.tools에서). 이 기능은 노트북 상단에서 가져온 것으로 GeoPandas의 기능과 동일하게 작동합니다.

즉,노트북 상단의 import 문을 변경하지 않고
아래 코드 셀에서 지오코딩 함수를 geocode()로 호출합니다.
코드가 의도한 대로 작동합니다!

In [52]:
# Create the geocoder
geolocator = Nominatim(user_agent="kaggle_learn")

def my_geocoder(row):
    point = geolocator.geocode(row).point
    return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})

berkeley_locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
starbucks.update(berkeley_locations)

#### 4.2. 버클리 위치 보기¶
방금 찾은 위치를 살펴보겠습니다. OpenStreetMap 스타일로 버클리의 (위도, 경도) 위치를 시각화합니다.

In [54]:
# Create a base map
m_2 = folium.Map(location=[37.88,-122.26], zoom_start=13)

# Your code here: Add a marker for each Berkeley location
# Add a marker for each Berkeley location
for idx, row in starbucks[starbucks["City"]=='Berkeley'].iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_2)


m_2

버클리의 5개 위치만 고려할 때 (위도, 경도) 위치가 잠재적으로 정확해 보이는 위치(올바른 도시에 위치)는 몇 개입니까?

=> 5개 모두 다 맞는 것 같습니다.

#### 4.3. 데이터를 통합합니다.
아래 코드를 실행하여 캘리포니아 주의 각 카운티에 대한 이름, 면적(제곱 킬로미터) 및 고유 ID("GEOID" 열에 있음)를 포함하는 GeoDataFrame CA_counties를 로드합니다. "형상" 열에는 카운티 경계가 있는 다각형이 포함되어 있습니다.

In [55]:
CA_counties = gpd.read_file("C:/Users/Kangdaeyong/Desktop/datamining/kaggle_geospatial_analysis/archive/CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.head()

,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


다음으로 3개의 DataFrame을 생성합니다.

CA_pop에는 각 카운티의 인구 추정치가 포함됩니다.
CA_high_earners에는 연간 소득이 $150,000 이상인 가구 수가 포함됩니다.
CA_median_age에는 각 카운티의 중간 연령이 포함됩니다.

In [56]:
CA_pop = pd.read_csv("C:/Users/Kangdaeyong/Desktop/datamining/kaggle_geospatial_analysis/archive/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("C:/Users/Kangdaeyong/Desktop/datamining/kaggle_geospatial_analysis/archive/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("C:/Users/Kangdaeyong/Desktop/datamining/kaggle_geospatial_analysis/archive/CA_county_median_age.csv", index_col="GEOID")

다음 코드 셀을 사용하여 CA_counties GeoDataFrame을 CA_pop, CA_high_earners 및 CA_median_age와 결합합니다.

결과 GeoDataFrame CA_stats의 이름을 지정하고 "GEOID", "name", "area_sqkm", "geometry", "population", "high_earners" 및 "median_age"의 8개 열이 있는지 확인합니다. 또한 CRS가 {'init': 'epsg:4326'}으로 설정되어 있는지 확인합니다.

In [71]:
cols_to_add = CA_pop.join([CA_high_earners, CA_median_age]).reset_index()
CA_stats = CA_counties.merge(cols_to_add, on="GEOID")
CA_stats.crs = {'init': 'epsg:4326'}

c:\Users\Kangdaeyong\anaconda3\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)



이제 모든 데이터가 한 곳에 있으므로 열 조합을 사용하는 통계를 훨씬 쉽게 계산할 수 있습니다. 다음 코드 셀을 실행하여 인구 밀도가 있는 "밀도" 열을 만듭니다.

In [72]:
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]

#### 4.4. 어느 카운티가 유망해 보입니까?
모든 정보를 단일 GeoDataFrame으로 축소하면 특정 기준을 충족하는 카운티를 훨씬 더 쉽게 선택할 수 있습니다.

다음 코드 셀을 사용하여 CA_stats GeoDataFrame에서 행(및 모든 열)의 하위 집합을 포함하는 GeoDataFrame sel_counties를 만듭니다. 특히 다음과 같은 카운티를 선택해야 합니다.
- 연간 $150,000를 버는 적어도 100,000 가구가 있고,
- 중위 연령이 38.5세 미만이고,
- 주민 밀도는 최소 285명(제곱 킬로미터당)입니다.
- 또한 선택한 카운티는 다음 기준 중 하나 이상을 충족해야 합니다.
- 연간 $150,000를 버는 적어도 500,000 가구가 있고,
- 중위 연령이 35.5세 미만이거나
- 주민 밀도는 최소 1400명(제곱 킬로미터당)입니다.

In [74]:
# Your code here
sel_counties = sel_counties = CA_stats[((CA_stats.high_earners > 100000) &
                         (CA_stats.median_age < 38.5) &
                         (CA_stats.density > 285) &
                         ((CA_stats.median_age < 35.5) |
                         (CA_stats.density > 1400) |
                         (CA_stats.high_earners > 500000)))]

sel_counties

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
5,6037,Los Angeles County,12305.376879,"MULTIPOLYGON (((-118.66761 33.47749, -118.6682...",10105518,501413,36.0,821.227834
8,6073,San Diego County,11721.342229,"POLYGON ((-117.43744 33.17953, -117.44955 33.1...",3343364,194676,35.4,285.237299
10,6075,San Francisco County,600.588247,"MULTIPOLYGON (((-122.60025 37.80249, -122.6123...",883305,114989,38.3,1470.733077


#### 4.5. 몇 개의 매장을 식별했습니까?
다음 Starbucks Reserve Roastery 위치를 찾을 때 선택한 카운티 내의 모든 매장을 고려하고 싶습니다. 그렇다면 선택한 카운티 내에 몇 개의 매장이 있습니까?

이 질문에 답할 준비를 하려면 다음 코드 셀을 실행하여 모든 스타벅스 위치가 포함된 GeoDataFrame starbucks_gdf를 만듭니다.

In [75]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}

c:\Users\Kangdaeyong\anaconda3\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


그렇다면 선택한 카운티에는 몇 개의 매장이 있습니까?

In [77]:
num_stores = locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
num_stores = len(locations_of_interest)
num_stores

1043

#### 4.6. 매장 위치를 ​​시각화합니다.
이전 질문에서 식별한 상점의 위치를 ​​보여주는 지도를 만드십시오.

In [82]:
# Create a base map
m_6 = folium.Map(location=[37,-120], zoom_start=6)

# Show selected store locations
mc = MarkerCluster()

locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
for idx, row in locations_of_interest.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))

m_6.add_child(mc)
